# 猫眼100
从多个网页 URL 中爬取电影信息，并将相关信息保存到本地

HTML 教程：https://www.runoob.com/html/html-tutorial.html

In [ ]:
<!-- 浏览器源代码与元素检查差异解析 -->

<!-- 源代码：从这里爬取内容 -->
<img data-src="https://p0.pipi.cn/mmdb/54ecde9a2c9f2a51ba06d67d795a9434b8421.jpg?imageView2/1/w/160/h/220" alt="我不是药神" class="board-img"/>

<!-- 检查→元素看到的：用于找位置、生成代码 -->
<img alt="我不是药神" class="board-img" src="https://p0.pipi.cn/mmdb/54ecde9a2c9f2a51ba06d67d795a9434b8421.jpg?imageView2/1/w/160/h/220">

1、清除网络日志

<img src="./images/maoyan01.png" width=800>

2、重新加载页面

<img src="./images/maoyan02.png" width=800>

3、查看页面内容

NOTE: 当前页面排在第 1 位，需要向上滚动滚动条才能看到

<img src="./images/maoyan03.png" width=800>

4、标头(确定请求 URL，请求方法（GET，POST），请求参数)

<img src="./images/maoyan04.png" width=800>

5、标头(请求标头，Cookies，User-Agent信息)

<img src="./images/maoyan05.png" width=500>
<img src="./images/maoyan06.png" width=500>

In [3]:
import os
import requests         # 请求模块
import re
import time
from lxml import etree  # 用于解析 HTML 文档
"""
1、明确需要爬的内容：电影名，主演，上映时间，评分，图片URL，保存图片
2、用Edge浏览器打开需要爬虫的网址，右键-检查-网络-清除网络日志-重新加载页面-点“名称”下面的内容-预览(判断我们需要爬取的内容在不在里面)
   -标头(确定请求URL，请求方法（GET，POST），请求参数，请求标头，Cookies，User-Agent信息)-翻页
"""

# https://www.maoyan.com/board/4?offset=0
# https://www.maoyan.com/board/4?offset=10
# ...
# https://www.maoyan.com/board/4?offset=90
url = 'https://www.maoyan.com/board/4?offset=0'
print(url)

https://www.maoyan.com/board/4?offset=0


In [4]:
urls = [f"https://www.maoyan.com/board/4?offset={page}" for page in range(0, 100, 10)]
print(urls)

['https://www.maoyan.com/board/4?offset=0', 'https://www.maoyan.com/board/4?offset=10', 'https://www.maoyan.com/board/4?offset=20', 'https://www.maoyan.com/board/4?offset=30', 'https://www.maoyan.com/board/4?offset=40', 'https://www.maoyan.com/board/4?offset=50', 'https://www.maoyan.com/board/4?offset=60', 'https://www.maoyan.com/board/4?offset=70', 'https://www.maoyan.com/board/4?offset=80', 'https://www.maoyan.com/board/4?offset=90']


In [5]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36 Edg/139.0.0.0",
           "Cookie": "__mta=45489947.1738158849724.1755570522414.1755573383891.237; _lxsdk_cuid=194b2573d2ec8-0d578b61f39bcd-4c657b58-144000-194b2573d2ec8; _ga=GA1.1.1071748017.1738158850; uuid_n_v=v1; uuid=733EF1C06C6F11F0AD0CB135C59C7EDB13E3C70B770D4B3E9C0C6E551BD4C15B; _csrf=b90f90cfbeb7f9e927fc8b1a4bc393b238b457010fdf3b3943275553c4e2d3c3; Hm_lvt_e0bacf12e04a7bd88ddbd9c74ef2b533=1752981812,1753261103,1754864811,1755128765; HMACCOUNT=8517ACA1A38CE54E; theme=moviepro; global-guide-isclose=true; __mta=45489947.1738158849724.1755505954658.1755562183276.506; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; _lxsdk=733EF1C06C6F11F0AD0CB135C59C7EDB13E3C70B770D4B3E9C0C6E551BD4C15B; Hm_lpvt_e0bacf12e04a7bd88ddbd9c74ef2b533=1755573384; _ga_WN80P4PSY7=GS2.1.s1755573383$o491$g1$t1755573383$j60$l0$h0; _lxsdk_s=198c053efca-a44-df6-845%7C%7C3"}

In [2]:
movie = '<a href="/films/790" title="蝙蝠侠：黑暗骑士" class="image-link" data-act="boarditem-click" data-val="{movieId:790}">'
# .：匹配除了换行符以外的任意一个字符 
# *?：对它前面的正则表达式匹配 0 到任意次重复，尽量少的匹配
res = re.findall(r'title="(.*?)"', movie)
print(res)

['蝙蝠侠：黑暗骑士']


In [6]:
# *：对它前面的正则表达式匹配 0 到任意次重复，尽量多的匹配
res = re.findall(r'title="(.*)"', movie)
print(res)

['蝙蝠侠：黑暗骑士" class="image-link" data-act="boarditem-click" data-val="{movieId:790}']


## 方法一：正则

In [ ]:
<!-- 源代码 -->
<dd>
    <i class="board-index board-index-1">1</i>
    <a href="/films/1200486" title="我不是药神" class="image-link" data-act="boarditem-click"
       data-val="{movieId:1200486}">
        <img src="//s3.meituan.net/static-prod01/com.sankuai.movie.fe.mywww-files/image/loading_2.e3d934bf.png" alt=""
             class="poster-default"/>
        <img data-src="https://p0.pipi.cn/mmdb/54ecde9a2c9f2a51ba06d67d795a9434b8421.jpg?imageView2/1/w/160/h/220"
             alt="我不是药神" class="board-img"/>
    </a>
    <div class="board-item-main">
        <div class="board-item-content">
            <div class="movie-item-info">
                <p class="name"><a href="/films/1200486" title="我不是药神" data-act="boarditem-click"
                                   data-val="{movieId:1200486}">我不是药神</a></p>
                <p class="star">
                    主演：徐峥,王传君,周一围
                </p>
                <p class="releasetime">上映时间：2018-07-05</p></div>
            <div class="movie-item-number score-num">
                <p class="score"><i class="integer">9.</i><i class="fraction">6</i></p>
            </div>

        </div>
    </div>
</dd>

提取 1 个 URL 中的内容：

In [9]:
# NOTE：如果没有爬到数据（因有反爬功能），需要刷新网页，并更新"Cookie"数据   

# 发送 GET 请求，获取 HTML 网页内容
res = requests.get(url, headers=headers)
res.encoding = 'utf-8'

# 提取电影信息
# (.*?)无法匹配换行符，所以要先去除换行符
# print(repr(res))
res = res.text.replace("\n", "")
# 提取所有 <dd>...</dd> 标签之间的内容
movies = re.findall(r"<dd>(.*?)</dd>", res)
# print(len(movies), movies)

# 保存原始 HTML 内容
# 返回的内容和我们看到的内容可能不一致，需要保存下来看一下
with open('./output/film.html', "w", encoding='utf-8') as f:
    f.write(res)

# 创建图片保存目录
dir_name = "film"
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

# 遍历每部电影信息
for movie in movies:
    # 名称、主演、上映时间、评分和图片 URL
    # .*?：非贪婪匹配
    name = re.findall(r'title="(.*?)"', movie)[0]  # (.*?)不排满，(.*)排满
    actor = re.findall(r'主演：(.*?)</p>', movie)[0]
    date = re.findall(r'上映时间：(.*?)</p>', movie)[0]
    score1 = re.findall(r'<i class="integer">(.*?)</i>', movie)[0]
    soore2 = re.findall(r'<i class="fraction">(.*?)</i>', movie)[0]
    score = score1 + soore2
    img_url = re.findall(r'data-src="(.*?)"', movie)[0]  # 需要根据修改后的名字 data-src，查找代码

    # 下载电影图片
    # 发送 GET 请求下载电影图片
    # 暂停 0.5 秒，避免请求过于频繁被服务器封禁
    img_res = requests.get(img_url, headers=headers)
    time.sleep(0.5)  

    # 保存图片到本地
    # 从 URL 中提取文件名（如 https://p0.pipi.cn/mmdb/image.jpg?... 提取为 image.jpg）
    file_name = img_url.split("?")[0].split("/")[-1]
    image_path = os.path.join(dir_name, file_name)
    
    # 以二进制保存
    with open(image_path, "wb") as f:
        f.write(img_res.content)

    # 打印电影信息
    print(name, actor, date, score, img_url)

蝙蝠侠：黑暗骑士 克里斯蒂安·贝尔,希斯·莱杰,阿伦·伊克哈特         2008-07-14(阿根廷) 9.3 https://p0.pipi.cn/mmdb/d2dad592c7e7e111e55bf165a69f4403a0815.jpg?imageView2/1/w/160/h/220
素媛 李来,薛景求,严智苑         2013-10-02(韩国) 9.1 https://p0.pipi.cn/mmdb/d2dad592b125bf230f8d3384ef8b5b7ad3de5.jpg?imageView2/1/w/160/h/220
两杆大烟枪 杰森·弗莱明,德克斯特·弗莱彻,杰森·斯坦森         1998-08-28(英国) 8.7 https://p0.pipi.cn/mmdb/54ecde02537ddd178967cbc55df9269f0bfee.jpg?imageView2/1/w/160/h/220
傲慢与偏见 马修·麦克费登,吉娜·马隆,妲露拉·莱莉         2008-02-10 8.4 https://p0.pipi.cn/mmdb/d2dad592b1251b2ff7be12b77068890525bd4.jpg?imageView2/1/w/160/h/220
奥本海默 基里安·墨菲,艾米莉·布朗特,马特·达蒙         2023-08-30 9.3 https://p0.pipi.cn/mmdb/54ecde8d537923be127a35edaa7ed1b006af8.jpg?imageView2/1/w/160/h/220
大鱼 伊万·麦克格雷格,阿尔伯特·芬尼,杰西卡·兰格         2003-12-04(美国) 8.6 https://p0.pipi.cn/mmdb/d2dad592c7e7e1d236d7c316db75fd5d48d41.jpg?imageView2/1/w/160/h/220
禁闭岛 莱昂纳多·迪卡普里奥,马克·鲁法洛,本·金斯利         2010-02-13(德国) 8.7 https://p0.pipi.cn/mmdb/d2dad592c7e7e111e5e19b6ea329f546d4160.jpg?imageView2/1/w/160/h/22

提取多个 URL 中的内容：

In [10]:
for url in urls:
    # 发送 GET 请求，获取 HTML 网页内容
    res = requests.get(url, headers=headers)
    res.encoding = 'utf-8'
    
    # 提取电影信息
    # (.*?)无法匹配换行符，所以要先去除换行符
    # print(repr(res))
    res = res.text.replace("\n", "")
    # 提取所有 <dd>...</dd> 标签之间的内容
    movies = re.findall(r"<dd>(.*?)</dd>", res)
    # print(len(movies), movies)
    # break
    
    # 保存原始 HTML 内容
    # 返回的内容和我们看到的内容可能不一致，需要保存下来看一下
    with open('./output/film.html', "w", encoding='utf-8') as f:
        f.write(res)

    # 创建图片保存目录
    dir_name = "film"
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    # 遍历每部电影信息
    for movie in movies:
        # 名称、主演、上映时间、评分和图片 URL
        # .*?：非贪婪匹配
        name = re.findall(r'title="(.*?)"', movie)[0]  # (.*?)不排满，(.*)排满
        actor = re.findall(r'主演：(.*?)</p>', movie)[0]
        date = re.findall(r'上映时间：(.*?)</p>', movie)[0]
        score1 = re.findall(r'<i class="integer">(.*?)</i>', movie)[0]
        soore2 = re.findall(r'<i class="fraction">(.*?)</i>', movie)[0]
        score = score1 + soore2
        img_url = re.findall(r'data-src="(.*?)"', movie)[0]  # 需要根据修改后的名字 data-src，查找代码

        # 下载电影图片
        # 发送 GET 请求下载电影图片
        # 暂停 0.5 秒，避免请求过于频繁被服务器封禁
        img_res = requests.get(img_url, headers=headers)
        time.sleep(0.5)  

        # 保存图片到本地
        # 从 URL 中提取文件名（如 https://p0.pipi.cn/mmdb/image.jpg?... 提取为 image.jpg）
        file_name = img_url.split("?")[0].split("/")[-1]
        image_path = os.path.join(dir_name, file_name)
        
        # 以二进制保存
        with open(image_path, "wb") as f:
            f.write(img_res.content)

        # 打印电影信息
        print(name, actor, date, score, img_url)

我不是药神 徐峥,王传君,周一围         2018-07-05 9.6 https://p0.pipi.cn/mmdb/54ecde9a2c9f2a51ba06d67d795a9434b8421.jpg?imageView2/1/w/160/h/220
肖申克的救赎 蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿         1994-09-10(加拿大) 9.5 https://p0.pipi.cn/mmdb/54ecde3311ef2abe12f0ee281b37b5fc0bc2b.jpg?imageView2/1/w/160/h/220
海上钢琴师 蒂姆·罗斯,比尔·努恩 ,克兰伦斯·威廉姆斯三世         2019-11-15 9.3 https://p0.pipi.cn/mediaplus/friday_image_fe/cdf05c5cf87ea4f87bd8189dd2c5c8c208886.jpg?imageView2/1/w/160/h/220
绿皮书 维果·莫腾森,马赫沙拉·阿里,琳达·卡德里尼         2019-03-01 9.5 https://p0.pipi.cn/mmdb/d2dad59253751b230f21f0818a5bfd4d8679c.jpg?imageView2/1/w/160/h/220
霸王别姬 张国荣,张丰毅,巩俐         1993-07-26 9.4 https://p0.pipi.cn/mmdb/fb7386beddd338537c8ea3bb80d25a9078b13.jpg?imageView2/1/w/160/h/220
美丽人生 罗伯托·贝尼尼,朱斯蒂诺·杜拉诺,赛尔乔·比尼·布斯特里克         2020-01-03 9.3 https://p0.pipi.cn/mmdb/d2dad592c7e7e1d2367a3507befaed31a5903.jpg?imageView2/1/w/160/h/220
这个杀手不太冷 让·雷诺,娜塔莉·波特曼,加里·奥德曼         2024-11-01 9.6 https://p0.pipi.cn/mmdb/54ecde92339923b12d807750dae3398ce9cce.jpg?imageView2/1/w/160/h

## 方法二：Xpath

In [21]:
for url in urls:
    res = requests.get(url, headers=headers)
    res.encoding = 'utf-8'
    xpath_html = etree.HTML(res.text)
    # time.sleep(1)
    movies = xpath_html.xpath('//*[@id="app"]/div/div/div[1]/dl/dd')
    
    for movie in movies:
        name = movie.xpath("./div/div/div[1]/p[1]/a/text()")
        actor = movie.xpath("./div/div/div[1]/p[2]/text()")
        img_url = movie.xpath("./a/img[2]/@data-src")
        print(name, actor, img_url)

['我不是药神'] ['\n                主演：徐峥,王传君,周一围\n        '] ['https://p0.pipi.cn/mmdb/54ecde9a2c9f2a51ba06d67d795a9434b8421.jpg?imageView2/1/w/160/h/220']
['肖申克的救赎'] ['\n                主演：蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿\n        '] ['https://p0.pipi.cn/mmdb/54ecde3311ef2abe12f0ee281b37b5fc0bc2b.jpg?imageView2/1/w/160/h/220']
['海上钢琴师'] ['\n                主演：蒂姆·罗斯,比尔·努恩 ,克兰伦斯·威廉姆斯三世\n        '] ['https://p0.pipi.cn/mediaplus/friday_image_fe/cdf05c5cf87ea4f87bd8189dd2c5c8c208886.jpg?imageView2/1/w/160/h/220']
['绿皮书'] ['\n                主演：维果·莫腾森,马赫沙拉·阿里,琳达·卡德里尼\n        '] ['https://p0.pipi.cn/mmdb/d2dad59253751b230f21f0818a5bfd4d8679c.jpg?imageView2/1/w/160/h/220']
['霸王别姬'] ['\n                主演：张国荣,张丰毅,巩俐\n        '] ['https://p0.pipi.cn/mmdb/fb7386beddd338537c8ea3bb80d25a9078b13.jpg?imageView2/1/w/160/h/220']
['美丽人生'] ['\n                主演：罗伯托·贝尼尼,朱斯蒂诺·杜拉诺,赛尔乔·比尼·布斯特里克\n        '] ['https://p0.pipi.cn/mmdb/d2dad592c7e7e1d2367a3507befaed31a5903.jpg?imageView2/1/w/160/h/220']
['这个杀手不太冷'] ['\n      